In [1]:
!pip cache purge

!pip install python-dotenv
!pip install httpx==0.27.2
!pip install anthropic
!pip install gradio
!pip install torch
!pip install -q transformers datasets diffusers
!pip install -q requests bitsandbytes accelerate sentencepiece
!pip install huggingface_hub

Files removed: 0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.9/207.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
from google.colab import userdata
drive.mount('/content/drive')

import torch
import transformers
import torchvision
from transformers import pipeline
from diffusers import DiffusionPipeline
from datasets import load_dataset
import soundfile as sf
from IPython.display import Audio

from diffusers import FluxPipeline

import anthropic
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import google.generativeai

import json
import gradio as gr
import time
import subprocess

import requests
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig, AutoModelForSpeechSeq2Seq, AutoProcessor

Mounted at /content/drive


In [3]:
from google.colab import files
uploaded = files.upload()

Saving API Key.env to API Key.env


In [4]:
load_dotenv("/content/API Key.env")
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('CLAUDE_API_KEY')
google_api_key = os.getenv('GEMINI_API_KEY')
hf_token = os.getenv('HF_TOKEN')

check_api_open = f"OpenAI API Key exists and begins: {openai_api_key[:8]}" if openai_api_key else "OpenAI API Key not set"
check_api_anthropic = f"Anthropic API Key exists and begins: {anthropic_api_key[:7]}" if anthropic_api_key else "Anthropic API Key not set"
check_api_gemini = f"Google API Key exists and begins: {google_api_key[:8]}" if google_api_key else "Google API Key not set"
check_api_hugging = f"Hugging API Key exists and begins: {hf_token[:8]}" if hf_token else "Hugging API Key not set"

display(check_api_open, check_api_anthropic, check_api_gemini, check_api_hugging)

'OpenAI API Key exists and begins: sk-proj-'

'Anthropic API Key exists and begins: sk-ant-'

'Google API Key exists and begins: AIzaSyCH'

'Hugging API Key exists and begins: hf_FoxyK'

In [5]:
openai = OpenAI(api_key = openai_api_key)
claude = anthropic.Anthropic(api_key=anthropic_api_key)
google.generativeai.configure(api_key = google_api_key)

In [6]:
system_message = "You are an assistant that reimplements Python code in high performance C++ for an windows. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time. Keep implementations of random number generators identical so that results match exactly."

In [7]:
def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    user_prompt += python
    return user_prompt


def user_prompt_for_testcase(python):
    user_prompt = "Write unit test cases for the C++ code to ensure it produces identical output to the original Python code. "
    user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    user_prompt += "Include all necessary C++ packages such as iostream, assert, and any other required libraries. "
    user_prompt += "Ensure that test cases cover a wide range of inputs and edge cases..\n\n"
    user_prompt += python
    return user_prompt

In [8]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

def messages_for_testcase(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for_testcase(python)}
    ]

# **Convert python to C++ (more model)**

In [9]:
# GPT
def convert_python_cpp_GPT(python):
    stream = openai.chat.completions.create(model="gpt-4o", messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
    reply = reply.strip()
    reply = reply.replace("```cpp", "").replace("```", "").strip()
    return reply


def convert_python_cpp_GPT_testcase(python):
    stream = openai.chat.completions.create(model="gpt-4o", messages=messages_for_testcase(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
    reply = reply.strip()
    reply = reply.replace("```cpp", "").replace("```", "").strip()
    return reply

In [10]:
# Claude
def convert_python_cpp_CLAUDE(python):
    result = claude.messages.stream(
        model="claude-3-5-sonnet-20241022",
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
    reply = reply.strip()
    reply = reply.replace("```cpp", "").replace("```", "").strip()
    return reply


def convert_python_cpp_CLAUDE_testcase(python):
    result = claude.messages.stream(
        model="claude-3-5-sonnet-20241022",
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for_testcase(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
    reply = reply.strip()
    reply = reply.replace("```cpp", "").replace("```", "").strip()
    return reply

In [11]:
# Gemini
import logging
def convert_python_cpp_GEMINI(python_code):
    """
    Chuyển đổi mã Python sang C++ sử dụng mô hình Gemini.

    :param python_code: Chuỗi mã Python cần chuyển đổi.
    :return: Mã C++ đã chuyển đổi.
    """
    try:
        logging.debug("Đang kết nối với Gemini...")
        model = google.generativeai.GenerativeModel("gemini-1.5-pro")

        logging.debug("Đang gửi yêu cầu chuyển đổi mã...")
        start_time = time.time()
        response = model.start_chat(
            history=[
                {"role": "model", "parts": system_message},
            ]
        ).send_message(user_prompt_for(python_code), stream=True)

        reply = ""
        for chunk in response:
            if time.time() - start_time > 60:
                return "Lỗi: Gemini không phản hồi kịp thời."
            reply += chunk.text

        logging.debug("Đã nhận phản hồi từ Gemini.")
        if "```cpp" in reply and "```" in reply:
            start_idx = reply.find("```cpp") + len("```cpp")
            end_idx = reply.rfind("```")
            cpp_code = reply[start_idx:end_idx].strip()
        else:
            cpp_code = reply.strip()
        return cpp_code
    except Exception as e:
        logging.error(f"Lỗi khi chuyển đổi mã: {e}")
        return f"Lỗi khi chuyển đổi mã: {e}"


def convert_python_cpp_GEMINI_testcase(python):
    model = google.generativeai.GenerativeModel("gemini-1.5-pro")
    response = model.start_chat(
        history=[
            {"role": "model", "parts": system_message},
        ]
    )

    response = response.send_message(user_prompt_for_testcase(python), stream=True)
    reply = ""
    for chunk in response:
        reply += chunk.text
    reply = reply.strip()
    reply = reply.replace("```cpp", "").replace("```", "").strip()
    return reply

In [12]:
print(convert_python_cpp_GPT("print('hello world')"))
print("\n")
print("\n")
print(convert_python_cpp_GPT_testcase("print('hello world')"))

#include <iostream>

int main() {
    std::cout << "hello world" << std::endl; // Output "hello world"
    return 0;
}




#include <iostream>
#include <cassert>

// Function that produces the same output as the Python code
void printHelloWorld() {
    std::cout << "hello world" << std::endl;
}

// Unit test function
void testPrintHelloWorld() {
    // Redirect std::cout to a string stream to capture the output
    std::streambuf* originalCoutStreamBuf = std::cout.rdbuf();
    std::ostringstream strCout;
    std::cout.rdbuf(strCout.rdbuf());

    // Call the function to test
    printHelloWorld();

    // Restore the original std::cout stream buffer
    std::cout.rdbuf(originalCoutStreamBuf);

    // Assert the captured output
    assert(strCout.str() == "hello world\n");
}

int main() {
    // Running the unit test
    testPrintHelloWorld();
    std::cout << "All tests passed!" << std::endl;
    return 0;
}


In [13]:
print(convert_python_cpp_CLAUDE("print('hello world')"))
print("\n")
print("\n")
print(convert_python_cpp_CLAUDE_testcase("print('hello world')"))

#include <iostream>

int main() {
    std::cout << "hello world\n";
    return 0;
}




#include <iostream>
#include <cassert>
#include <sstream>

class OutputCapture {
    std::stringstream buffer;
    std::streambuf* old;
public:
    OutputCapture() {
        old = std::cout.rdbuf(buffer.rdbuf());
    }
    ~OutputCapture() {
        std::cout.rdbuf(old);
    }
    std::string getOutput() {
        return buffer.str();
    }
};

void test_hello_world() {
    OutputCapture capture;
    std::cout << "hello world" << std::endl;
    assert(capture.getOutput() == "hello world\n");
}

int main() {
    test_hello_world();
    std::cout << "All tests passed!" << std::endl;
    return 0;
}


In [14]:
print(convert_python_cpp_GEMINI("print('hello world')"))
print("\n")
print("\n")
print(convert_python_cpp_GEMINI_testcase("print('hello world')"))

#include <iostream>
#include <iomanip>

int main() {
  std::cout << "hello world" << std::endl;
  return 0;
}




#include <iostream>
#include <string>
#include <cassert>

int main() {
    // Test case 1: Basic output
    std::string expected_output1 = "hello world\n";
    std::streambuf* old_cout = std::cout.rdbuf(); // Redirect cout
    std::ostringstream test_cout;
    std::cout.rdbuf(test_cout.rdbuf());

    std::cout << "hello world" << std::endl; // Replace with function call

    std::cout.rdbuf(old_cout); // Restore cout
    assert(test_cout.str() == expected_output1);
     std::cout << "Test case 1 passed" << std::endl;


    return 0;
}


In [15]:
model = google.generativeai.GenerativeModel("gemini-1.5-flash")
response = model.generate_content("Explain how AI works", stream=True)
for chunk in response:
    print(chunk.text, end="")

Artificial intelligence (AI) is a broad field encompassing many techniques, but at its core, it's about creating systems that can perform tasks that typically require human intelligence.  There's no single "how it works" answer, as different AI approaches use different methods. However, we can break down the general principles:

**1. Data is King:**  AI systems learn from data.  The more relevant and high-quality data they are trained on, the better they perform.  This data can be anything from images and text to sensor readings and financial transactions.

**2. Algorithms as the Engine:**  Algorithms are sets of rules and statistical techniques that AI systems use to process and learn from data.  These algorithms allow the system to identify patterns, make predictions, and improve its performance over time.  Key algorithm types include:

* **Machine Learning (ML):**  This is a subset of AI where systems learn from data without explicit programming.  Instead of being explicitly told wh

In [16]:
model = google.generativeai.GenerativeModel("gemini-1.5-flash")
chat = model.start_chat(
    history=[
        {"role": "user", "parts": "Hello"},
        {"role": "model", "parts": "Great to meet you. What would you like to know?"},
    ]
)

response = chat.send_message("I have 2 dogs in my house.", stream=True)
for chunk in response:
    print(chunk.text, end="")

That's lovely!  What breeds are they?  Do you have any fun stories about them?


In [17]:
python_code = """
# Danh sách các minterm (các tổ hợp đầu vào mà hàm Boolean trả về 1)
minterms = [0, 1, 2, 5, 6, 7]

# Số biến đầu vào
num_variables = 3

# Hàm để kiểm tra xem hai số nhị phân có khác nhau đúng 1 bit không
def is_adjacent(a, b):
    diff = a ^ b
    return (diff & (diff - 1)) == 0

# Tạo danh sách các nhóm dựa trên số lượng bit 1
groups = {}
for minterm in minterms:
    count = bin(minterm).count('1')
    if count not in groups:
        groups[count] = []
    groups[count].append((minterm, [minterm]))

# Tìm các tổ hợp có thể kết hợp
prime_implicants = []
while True:
    new_groups = {}
    merged = set()
    keys = sorted(groups.keys())
    for i in range(len(keys) - 1):
        for a in groups[keys[i]]:
            for b in groups[keys[i + 1]]:
                if is_adjacent(a[0], b[0]):
                    merged.add(a[0])
                    merged.add(b[0])
                    combined = (a[0], a[1] + b[1])
                    count = bin(combined[0]).count('1')
                    if count not in new_groups:
                        new_groups[count] = []
                    new_groups[count].append(combined)
    for key in groups:
        for a in groups[key]:
            if a[0] not in merged:
                prime_implicants.append(a)
    if not new_groups:
        break
    groups = new_groups

# In kết quả (sẽ không hiển thị do stdout bị chuyển hướng)
print("Các prime implicants:")
for pi in prime_implicants:
    print(f"Minterms: {pi[1]} -> Binary: {format(pi[0], f'0{num_variables}b')}")
"""

result = convert_python_cpp_GEMINI(python_code)
print(result)

#include <iostream>
#include <vector>
#include <unordered_map>
#include <algorithm>
#include <cmath>
#include <bitset>
#include <iomanip>

using namespace std;

// Function to check if two binary numbers differ by exactly one bit
bool is_adjacent(int a, int b) {
    int diff = a ^ b;
    return (diff & (diff - 1)) == 0;
}

int main() {
    vector<int> minterms = {0, 1, 2, 5, 6, 7};
    int num_variables = 3;

    unordered_map<int, vector<pair<int, vector<int>>>> groups;
    for (int minterm : minterms) {
        int count = __builtin_popcount(minterm); // Use builtin popcount for speed
        groups[count].push_back({minterm, {minterm}});
    }

    vector<pair<int, vector<int>>> prime_implicants;
    while (true) {
        unordered_map<int, vector<pair<int, vector<int>>>> new_groups;
        unordered_set<int> merged;
        vector<int> keys;
        for (auto const& [key, val] : groups) {
            keys.push_back(key);
        }
        sort(keys.begin(), keys.end());

        

# **Calculate time**

In [18]:
import time
import sys
from io import StringIO
import io

def execute_python_code(code):
    """
    Thực thi một đoạn mã Python được truyền vào dưới dạng chuỗi.

    :param code: Chuỗi mã Python cần thực thi.
    :return: Kết quả thực thi, thời gian thực thi và đầu ra của mã.
    """
    start_time = time.time()

    old_stdout = sys.stdout
    old_stderr = sys.stderr
    new_stdout = io.StringIO()
    new_stderr = io.StringIO()
    sys.stdout = new_stdout
    sys.stderr = new_stderr

    try:
        exec(code)
    except Exception as e:
        error_message = f"Execution Error: {e}\n{new_stderr.getvalue()}"
        return f"Python Execution Time: {time.time() - start_time:.6f} seconds", error_message
    finally:
        sys.stdout = old_stdout
        sys.stderr = old_stderr

    end_time = time.time()
    execution_time = end_time - start_time
    output = new_stdout.getvalue()
    error_output = new_stderr.getvalue()

    if error_output:
        return f"Python Execution Time: {execution_time:.6f} seconds"
    else:
        return f"Python Execution Time: {execution_time:.6f} seconds"

python_code = """
# Danh sách các minterm (các tổ hợp đầu vào mà hàm Boolean trả về 1)
minterms = [0, 1, 2, 5, 6, 7]

# Số biến đầu vào
num_variables = 3

# Hàm để kiểm tra xem hai số nhị phân có khác nhau đúng 1 bit không
def is_adjacent(a, b):
    diff = a ^ b
    return (diff & (diff - 1)) == 0

# Tạo danh sách các nhóm dựa trên số lượng bit 1
groups = {}
for minterm in minterms:
    count = bin(minterm).count('1')
    if count not in groups:
        groups[count] = []
    groups[count].append((minterm, [minterm]))

# Tìm các tổ hợp có thể kết hợp
prime_implicants = []
while True:
    new_groups = {}
    merged = set()
    keys = sorted(groups.keys())
    for i in range(len(keys) - 1):
        for a in groups[keys[i]]:
            for b in groups[keys[i + 1]]:
                if is_adjacent(a[0], b[0]):
                    merged.add(a[0])
                    merged.add(b[0])
                    combined = (a[0], a[1] + b[1])
                    count = bin(combined[0]).count('1')
                    if count not in new_groups:
                        new_groups[count] = []
                    new_groups[count].append(combined)
    for key in groups:
        for a in groups[key]:
            if a[0] not in merged:
                prime_implicants.append(a)
    if not new_groups:
        break
    groups = new_groups

# In kết quả (sẽ không hiển thị do stdout bị chuyển hướng)
print("Các prime implicants:")
for pi in prime_implicants:
    print(f"Minterms: {pi[1]} -> Binary: {format(pi[0], f'0{num_variables}b')}")
"""

result = execute_python_code(python_code)
print(result)

Python Execution Time: 0.000655 seconds


In [19]:
def compile_and_run_cpp(code):
    """
    Biên dịch và chạy mã C++.

    :param code: Chuỗi mã C++ cần thực thi.
    :return: Kết quả thực thi và thời gian thực thi.
    """

    cpp_filename = "temp.cpp"
    exe_filename = "temp"
    with open(cpp_filename, "w") as f:
        f.write(code)

    compile_command = f"g++ {cpp_filename} -o {exe_filename}"
    compile_process = subprocess.run(compile_command, shell=True, capture_output=True, text=True)
    if compile_process.returncode != 0:
        return f"Compilation Error: {compile_process.stderr}"

    start_time = time.time()
    run_process = subprocess.run(f"./{exe_filename}", shell=True, capture_output=True, text=True)
    end_time = time.time()

    os.remove(cpp_filename)
    os.remove(exe_filename)

    execution_time = end_time - start_time
    return f"C++ Execution Time: {execution_time} seconds"


cpp_code = convert_python_cpp_GPT("print('hello world')")
print(compile_and_run_cpp(cpp_code))

C++ Execution Time: 0.0033235549926757812 seconds


In [20]:
def choose_model(python, model_name):
    if model_name == "GPT":
        conversion_function = convert_python_cpp_GPT
        test_case_function = convert_python_cpp_GPT_testcase
    elif model_name == "Claude":
        conversion_function = convert_python_cpp_CLAUDE
        test_case_function = convert_python_cpp_CLAUDE_testcase
    elif model_name == "Gemini":
        conversion_function = convert_python_cpp_GEMINI
        test_case_function = convert_python_cpp_GEMINI_testcase
    else:
        raise ValueError("Invalid model selected")

    try:
        cpp_code = conversion_function(python)
        unit_test_case = test_case_function(python)
        return cpp_code, unit_test_case
    except Exception as e:
        logging.error(f"Lỗi trong choose_model: {e}")
        return f"Lỗi: {e}", ""

In [21]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

In [22]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
        test_case = gr.Textbox(label="Test case:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude", "Gemini"], label="Select model", value="GPT")
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ result:", elem_classes=["cpp"])

    convert.click(choose_model, inputs=[python, model], outputs=[cpp, test_case])
    python_run.click(execute_python_code, inputs=[python], outputs=[python_out])
    cpp_run.click(compile_and_run_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://66c0f87f21d248c3db.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
